In [16]:
import findspark
findspark.init()

from pyspark.sql import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window

import numpy as np
import pandas as pd

In [11]:
import sys
import os

#Need to set environment variables in order to make pyspark work in Python3.6
spark_home=os.environ['SPARK_HOME']
os.environ['PYTHONPATH']= spark_home+"/python/lib/py4j-0.10.4-src.zip"
os.environ['PYSPARK_PYTHON']="/dfm0/util/dfm_python/python36/bin/python3.6"
os.environ['PYSPARK_DRIVER_PYTHON']="/dfm0/util/dfm_python/python36/bin/python3.6"

super_dir='/dfm1/lijli06/DFM_SPARK'

In [12]:
spark = SparkSession.builder \
            .master("yarn") \
            .appName('get_mode_country_linhai') \
            .config("spark.executor.instances", 6) \
            .config("spark.executor.cores",5) \
            .config("spark.executor.memory",'29G') \
            .config("spark.submit.deployMode",'client') \
            .config("spark.executorEnv.PYTHONHASHSEED", 123) \
            .getOrCreate()

In [40]:
tmp_paths = "hdfs://dfm-cluster/DFM/sysauditlog/2018/08/*/sysauditlog.avro"

In [41]:
special_orgs = ['HSBCME', 'REVWORDWIDE', 'SEBCREDIT', 'SEBDEBIT', 'SEBCORPDEBIT']
orgs = pd.read_csv('/dfm1/lijli06/DFM_SPARK/jobs/HSBC_Group.txt',header=0)

In [43]:
spark_df = spark.read.format("com.databricks.spark.avro").load(tmp_paths)
# spark_df = spark_df.filter(spark_df.ORGNAME.isin(special_orgs)).select('ORGNAME', 'FIBIN', 'COUNTRY')
spark_df = spark_df.filter(spark_df.ORGNAME.isin(orgs['ORGNAME'].tolist()) & (spark_df.ISSUERCOUNTRY != '') & (spark_df.ISSUERCOUNTRY.isNotNull()))
spark_df = spark_df.select('ORGNAME', 'COUNTRYISOCODE')

In [44]:
bin_country = spark_df.rdd.map(lambda x: ((x.ORGNAME, x.COUNTRYISOCODE),1)).reduceByKey(lambda x,y: x+y) \
                      .map(lambda x: (x[0][0], [x[0][1], x[1]])).reduceByKey(lambda x,y: x if x[1]>y[1] else y) \
                      .map(lambda x: [x[0], x[1][0]]).collect()

In [ ]:
bin_country_df = pd.DataFrame(bin_country, columns=['ORGNAME','ISSUERCOUNTRY'])
bin_country_df

In [46]:
ccodes = pd.read_csv("/dfm1/lijli06/DFM_SPARK/extn_hashes/country_codes.csv", header=0, keep_default_na=False)

In [47]:
for idx,row in bin_country_df.iterrows():
    isocode = row['ISSUERCOUNTRY']
    country = ccodes.loc[ccodes['alpha-2']==isocode, 'name'].tolist()
    if len(country)>0:
        bin_country_df.loc[idx, 'ISSUERCOUNTRY'] = country[0]
    else:
        print("{} not found".format(isocode))

In [66]:
spark.stop()